# pyrealsense를 활용해 point cloud와 color array를 저장합니다
## <사전 param 설정>

In [1]:
# License: Apache 2.0. See LICENSE file in root directory.
# Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

"""
OpenCV and Numpy Point cloud Software Renderer
This sample is mostly for demonstration and educational purposes.
It really doesn't offer the quality or performance that can be
achieved with hardware acceleration.
Usage:
------
Mouse: 
    Drag with left button to rotate around pivot (thick small axes), 
    with right button to translate and the wheel to zoom.
Keyboard: 
    [p]     Pause
    [r]     Reset View
    [d]     Cycle through decimation values
    [z]     Toggle point scaling
    [c]     Toggle color source
    [s]     Save PNG (./out.png)
    [e]     Export points to ply (./out.ply)
    [q\ESC] Quit
"""

import math
import time
import cv2
import numpy as np
import pyrealsense2 as rs
import os


class AppState:

    def __init__(self, *args, **kwargs):
        self.WIN_NAME = 'RealSense'
        self.pitch, self.yaw = math.radians(-10), math.radians(-15)
        self.translation = np.array([0, 0, -1], dtype=np.float32)
        self.distance = 2
        self.prev_mouse = 0, 0
        self.mouse_btns = [False, False, False]
        self.paused = False
        self.decimate = 1
        self.scale = True
        self.color = True

    def reset(self):
        self.pitch, self.yaw, self.distance = 0, 0, 2
        self.translation[:] = 0, 0, -1

    @property
    def rotation(self):
        Rx, _ = cv2.Rodrigues((self.pitch, 0, 0))
        Ry, _ = cv2.Rodrigues((0, self.yaw, 0))
        return np.dot(Ry, Rx).astype(np.float32)

    @property
    def pivot(self):
        return self.translation + np.array((0, 0, self.distance), dtype=np.float32)


state = AppState()

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get stream profile and camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height

# Processing blocks
pc = rs.pointcloud()
decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
colorizer = rs.colorizer()


def mouse_cb(event, x, y, flags, param):

    if event == cv2.EVENT_LBUTTONDOWN:
        state.mouse_btns[0] = True

    if event == cv2.EVENT_LBUTTONUP:
        state.mouse_btns[0] = False

    if event == cv2.EVENT_RBUTTONDOWN:
        state.mouse_btns[1] = True

    if event == cv2.EVENT_RBUTTONUP:
        state.mouse_btns[1] = False

    if event == cv2.EVENT_MBUTTONDOWN:
        state.mouse_btns[2] = True

    if event == cv2.EVENT_MBUTTONUP:
        state.mouse_btns[2] = False

    if event == cv2.EVENT_MOUSEMOVE:

        h, w = out.shape[:2]
        dx, dy = x - state.prev_mouse[0], y - state.prev_mouse[1]

        if state.mouse_btns[0]:
            state.yaw += float(dx) / w * 2
            state.pitch -= float(dy) / h * 2

        elif state.mouse_btns[1]:
            dp = np.array((dx / w, dy / h, 0), dtype=np.float32)
            state.translation -= np.dot(state.rotation, dp)

        elif state.mouse_btns[2]:
            dz = math.sqrt(dx**2 + dy**2) * math.copysign(0.01, -dy)
            state.translation[2] += dz
            state.distance -= dz

    if event == cv2.EVENT_MOUSEWHEEL:
        dz = math.copysign(0.1, flags)
        state.translation[2] += dz
        state.distance -= dz

    state.prev_mouse = (x, y)


cv2.namedWindow(state.WIN_NAME, cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow(state.WIN_NAME, w, h)
cv2.setMouseCallback(state.WIN_NAME, mouse_cb)


def project(v):
    """project 3d vector array to 2d"""
    h, w = out.shape[:2]
    view_aspect = float(h)/w

    # ignore divide by zero for invalid depth
    with np.errstate(divide='ignore', invalid='ignore'):
        proj = v[:, :-1] / v[:, -1, np.newaxis] * \
            (w*view_aspect, h) + (w/2.0, h/2.0)

    # near clipping
    znear = 0.03
    proj[v[:, 2] < znear] = np.nan
    return proj


def view(v):
    """apply view transformation on vector array"""
    return np.dot(v - state.pivot, state.rotation) + state.pivot - state.translation


def line3d(out, pt1, pt2, color=(0x80, 0x80, 0x80), thickness=1):
    """draw a 3d line from pt1 to pt2"""
    p0 = project(pt1.reshape(-1, 3))[0]
    p1 = project(pt2.reshape(-1, 3))[0]
    if np.isnan(p0).any() or np.isnan(p1).any():
        return
    p0 = tuple(p0.astype(int))
    p1 = tuple(p1.astype(int))
    rect = (0, 0, out.shape[1], out.shape[0])
    inside, p0, p1 = cv2.clipLine(rect, p0, p1)
    if inside:
        cv2.line(out, p0, p1, color, thickness, cv2.LINE_AA)


def grid(out, pos, rotation=np.eye(3), size=1, n=10, color=(0x80, 0x80, 0x80)):
    """draw a grid on xz plane"""
    pos = np.array(pos)
    s = size / float(n)
    s2 = 0.5 * size
    for i in range(0, n+1):
        x = -s2 + i*s
        line3d(out, view(pos + np.dot((x, 0, -s2), rotation)),
               view(pos + np.dot((x, 0, s2), rotation)), color)
    for i in range(0, n+1):
        z = -s2 + i*s
        line3d(out, view(pos + np.dot((-s2, 0, z), rotation)),
               view(pos + np.dot((s2, 0, z), rotation)), color)


def axes(out, pos, rotation=np.eye(3), size=0.075, thickness=2):
    """draw 3d axes"""
    line3d(out, pos, pos +
           np.dot((0, 0, size), rotation), (0xff, 0, 0), thickness)
    line3d(out, pos, pos +
           np.dot((0, size, 0), rotation), (0, 0xff, 0), thickness)
    line3d(out, pos, pos +
           np.dot((size, 0, 0), rotation), (0, 0, 0xff), thickness)


def frustum(out, intrinsics, color=(0x40, 0x40, 0x40)):
    """draw camera's frustum"""
    orig = view([0, 0, 0])
    w, h = intrinsics.width, intrinsics.height

    for d in range(1, 6, 2):
        def get_point(x, y):
            p = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], d)
            line3d(out, orig, view(p), color)
            return p

        top_left = get_point(0, 0)
        top_right = get_point(w, 0)
        bottom_right = get_point(w, h)
        bottom_left = get_point(0, h)

        line3d(out, view(top_left), view(top_right), color)
        line3d(out, view(top_right), view(bottom_right), color)
        line3d(out, view(bottom_right), view(bottom_left), color)
        line3d(out, view(bottom_left), view(top_left), color)


def pointcloud(out, verts, texcoords, color, painter=True):
    """draw point cloud with optional painter's algorithm"""
    if painter:
        # Painter's algo, sort points from back to front

        # get reverse sorted indices by z (in view-space)
        # https://gist.github.com/stevenvo/e3dad127598842459b68
        v = view(verts)
        s = v[:, 2].argsort()[::-1]
        proj = project(v[s])
    else:
        proj = project(view(verts))

    if state.scale:
        proj *= 0.5**state.decimate

    h, w = out.shape[:2]

    # proj now contains 2d image coordinates
    j, i = proj.astype(np.uint32).T

    # create a mask to ignore out-of-bound indices
    im = (i >= 0) & (i < h)
    jm = (j >= 0) & (j < w)
    m = im & jm

    cw, ch = color.shape[:2][::-1]
    if painter:
        # sort texcoord with same indices as above
        # texcoords are [0..1] and relative to top-left pixel corner,
        # multiply by size and add 0.5 to center
        v, u = (texcoords[s] * (cw, ch) + 0.5).astype(np.uint32).T
    else:
        v, u = (texcoords * (cw, ch) + 0.5).astype(np.uint32).T
    # clip texcoords to image
    np.clip(u, 0, ch-1, out=u)
    np.clip(v, 0, cw-1, out=v)

    # perform uv-mapping
    out[i[m], j[m]] = color[u[m], v[m]]



out = np.empty((h, w, 3), dtype=np.uint8)

def get_verts(name):
    npsavePath='C://Users//user//Desktop//openPose//pointcloud/'+name
    pc_token = np.load(npsavePath)
    return pc_token

def zero_filter(x):
    if (x[2] > 1.5) and (x[2] < 3):
        return [x[0],x[1],x[2]]
    else:
        return [0,0,0]
    
def binary(x):
    if (x == 0):
        return 0
    else:
        return 0.5
   

In [2]:
framecount =0

# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)


Depth Scale is:  0.0010000000474974513


## <데이터 저장>

In [3]:
i=0
sums=np.zeros((76800, 3))
u_sum=np.zeros((76800, 3))
framecount =0

while True:
    # Grab camera data
    if not state.paused:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_frame = decimate.process(depth_frame)

        # Grab new intrinsics (may be changed by decimation)
        depth_intrinsics = rs.video_stream_profile(
            depth_frame.profile).get_intrinsics()
        w, h = depth_intrinsics.width, depth_intrinsics.height

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        depth_colormap = np.asanyarray(
            colorizer.colorize(depth_frame).get_data())

        if state.color:
            mapped_frame, color_source = color_frame, color_image
        else:
            mapped_frame, color_source = depth_frame, depth_colormap
        
        points = pc.calculate(depth_frame)
        pc.map_to(mapped_frame)
        
        # Pointcloud data to arrays
        v, t = points.get_vertices(), points.get_texture_coordinates()
        verts = np.asanyarray(v).view(np.float32).reshape(-1, 3)  # xyz
        npsavePath='C://Users//user//Desktop//openPose//outputs//pointcloud/'+str(framecount).zfill(5)
        np.save(npsavePath,verts)
        
        texcoords = np.asanyarray(t).view(np.float32).reshape(-1, 2)  # uv
        tcsavePath='C://Users//user//Desktop//openPose//outputs//texcoords/'+str(framecount)
        #np.save(tcsavePath,texcoords)
        
        plysavePath='C://Users//user//Desktop//openPose//outputs//ply/'+str(framecount)+'.ply'
        #points.export_to_ply(plysavePath, mapped_frame)
        
        ###############################################
        
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            print('Error, fail to get depth or color frame')
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        savePath_d = 'C://Users//user//Desktop//openPose//outputs//test_depth_array/' + str(framecount).zfill(5)
        savePath_c = 'C://Users//user//Desktop//openPose//outputs//test_color_array/' + str(framecount).zfill(5)
        np.save(savePath_d,depth_image)
        np.save(savePath_c,color_image)#1개의 배열을 NumPy format의 바이너리 파일로 저장하기 (Save a single array to a binary file in NumPy format)
        
        # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((color_image, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        
        key = cv2.waitKey(1)
        print('frame : ',framecount)
        print('time : 'framecount/30,'sec')
        framecount = framecount+1
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break

# Stop streaming

frame :  0
frame :  1
frame :  2
frame :  3
frame :  4
frame :  5
frame :  6
frame :  7
frame :  8
frame :  9
frame :  10
frame :  11
frame :  12
frame :  13
frame :  14
frame :  15
frame :  16
frame :  17
frame :  18
frame :  19
frame :  20
frame :  21
frame :  22
frame :  23
frame :  24
frame :  25
frame :  26
frame :  27
frame :  28
frame :  29
frame :  30
frame :  31
frame :  32
frame :  33
frame :  34
frame :  35
frame :  36
frame :  37
frame :  38
frame :  39
frame :  40
frame :  41
frame :  42
frame :  43
frame :  44
frame :  45
frame :  46
frame :  47
frame :  48
frame :  49
frame :  50
frame :  51
frame :  52
frame :  53
frame :  54
frame :  55
frame :  56
frame :  57
frame :  58
frame :  59
frame :  60
frame :  61
frame :  62
frame :  63
frame :  64
frame :  65
frame :  66
frame :  67
frame :  68
frame :  69
frame :  70
frame :  71
frame :  72
frame :  73
frame :  74
frame :  75
frame :  76
frame :  77
frame :  78
frame :  79
frame :  80
frame :  81
frame :  82
frame :  83
fr

frame :  644
frame :  645
frame :  646
frame :  647
frame :  648
frame :  649
frame :  650
frame :  651
frame :  652
frame :  653
frame :  654
frame :  655
frame :  656
frame :  657
frame :  658
frame :  659
frame :  660
frame :  661
frame :  662
frame :  663
frame :  664
frame :  665
frame :  666
frame :  667
frame :  668
frame :  669
frame :  670
frame :  671
frame :  672
frame :  673
frame :  674
frame :  675
frame :  676
frame :  677
frame :  678
frame :  679
frame :  680
frame :  681
frame :  682
frame :  683
frame :  684
frame :  685
frame :  686
frame :  687
frame :  688
frame :  689
frame :  690
frame :  691
frame :  692
frame :  693
frame :  694
frame :  695
frame :  696
frame :  697
frame :  698
frame :  699
frame :  700
frame :  701
frame :  702
frame :  703
frame :  704
frame :  705
frame :  706
frame :  707
frame :  708
frame :  709
frame :  710
frame :  711
frame :  712
frame :  713
frame :  714
frame :  715
frame :  716
frame :  717
frame :  718
frame :  719
frame :  720

# 저장한 array를 color image와 video로 변환합니다

In [5]:
import os
import numpy as np
import cv2

videopath = 'C://Users//user//Desktop//openPose//outputs//test_color_array/'
frame_list = sorted(os.listdir(videopath)) #피험자 리스트
print(frame_list)

outputVideo = []

for j in frame_list :#frame별 slice
    open_frame = videopath +'//' + j
    save_load = np.load(open_frame)
    imagepath = 'C://Users//user//Desktop//openPose//outputs//frames/'
    imagename = j+'.jpg'
    cv2.imwrite(os.path.join(imagepath ,imagename), save_load)
    print(j,save_load.shape)
    outputVideo.append(save_load)


out = cv2.VideoWriter('C://Users/user/Desktop/openPose//outputs/raw_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (640,480))

for i in range(len(outputVideo)):
    out.write(outputVideo[i])
              
out.release()

['00000.npy', '00001.npy', '00002.npy', '00003.npy', '00004.npy', '00005.npy', '00006.npy', '00007.npy', '00008.npy', '00009.npy', '00010.npy', '00011.npy', '00012.npy', '00013.npy', '00014.npy', '00015.npy', '00016.npy', '00017.npy', '00018.npy', '00019.npy', '00020.npy', '00021.npy', '00022.npy', '00023.npy', '00024.npy', '00025.npy', '00026.npy', '00027.npy', '00028.npy', '00029.npy', '00030.npy', '00031.npy', '00032.npy', '00033.npy', '00034.npy', '00035.npy', '00036.npy', '00037.npy', '00038.npy', '00039.npy', '00040.npy', '00041.npy', '00042.npy', '00043.npy', '00044.npy', '00045.npy', '00046.npy', '00047.npy', '00048.npy', '00049.npy', '00050.npy', '00051.npy', '00052.npy', '00053.npy', '00054.npy', '00055.npy', '00056.npy', '00057.npy', '00058.npy', '00059.npy', '00060.npy', '00061.npy', '00062.npy', '00063.npy', '00064.npy', '00065.npy', '00066.npy', '00067.npy', '00068.npy', '00069.npy', '00070.npy', '00071.npy', '00072.npy', '00073.npy', '00074.npy', '00075.npy', '00076.npy'

00016.npy (480, 640, 3)
00017.npy (480, 640, 3)
00018.npy (480, 640, 3)
00019.npy (480, 640, 3)
00020.npy (480, 640, 3)
00021.npy (480, 640, 3)
00022.npy (480, 640, 3)
00023.npy (480, 640, 3)
00024.npy (480, 640, 3)
00025.npy (480, 640, 3)
00026.npy (480, 640, 3)
00027.npy (480, 640, 3)
00028.npy (480, 640, 3)
00029.npy (480, 640, 3)
00030.npy (480, 640, 3)
00031.npy (480, 640, 3)
00032.npy (480, 640, 3)
00033.npy (480, 640, 3)
00034.npy (480, 640, 3)
00035.npy (480, 640, 3)
00036.npy (480, 640, 3)
00037.npy (480, 640, 3)
00038.npy (480, 640, 3)
00039.npy (480, 640, 3)
00040.npy (480, 640, 3)
00041.npy (480, 640, 3)
00042.npy (480, 640, 3)
00043.npy (480, 640, 3)
00044.npy (480, 640, 3)
00045.npy (480, 640, 3)
00046.npy (480, 640, 3)
00047.npy (480, 640, 3)
00048.npy (480, 640, 3)
00049.npy (480, 640, 3)
00050.npy (480, 640, 3)
00051.npy (480, 640, 3)
00052.npy (480, 640, 3)
00053.npy (480, 640, 3)
00054.npy (480, 640, 3)
00055.npy (480, 640, 3)
00056.npy (480, 640, 3)
00057.npy (480, 

00367.npy (480, 640, 3)
00368.npy (480, 640, 3)
00369.npy (480, 640, 3)
00370.npy (480, 640, 3)
00371.npy (480, 640, 3)
00372.npy (480, 640, 3)
00373.npy (480, 640, 3)
00374.npy (480, 640, 3)
00375.npy (480, 640, 3)
00376.npy (480, 640, 3)
00377.npy (480, 640, 3)
00378.npy (480, 640, 3)
00379.npy (480, 640, 3)
00380.npy (480, 640, 3)
00381.npy (480, 640, 3)
00382.npy (480, 640, 3)
00383.npy (480, 640, 3)
00384.npy (480, 640, 3)
00385.npy (480, 640, 3)
00386.npy (480, 640, 3)
00387.npy (480, 640, 3)
00388.npy (480, 640, 3)
00389.npy (480, 640, 3)
00390.npy (480, 640, 3)
00391.npy (480, 640, 3)
00392.npy (480, 640, 3)
00393.npy (480, 640, 3)
00394.npy (480, 640, 3)
00395.npy (480, 640, 3)
00396.npy (480, 640, 3)
00397.npy (480, 640, 3)
00398.npy (480, 640, 3)
00399.npy (480, 640, 3)
00400.npy (480, 640, 3)
00401.npy (480, 640, 3)
00402.npy (480, 640, 3)
00403.npy (480, 640, 3)
00404.npy (480, 640, 3)
00405.npy (480, 640, 3)
00406.npy (480, 640, 3)
00407.npy (480, 640, 3)
00408.npy (480, 

00721.npy (480, 640, 3)
00722.npy (480, 640, 3)
00723.npy (480, 640, 3)
00724.npy (480, 640, 3)
00725.npy (480, 640, 3)
00726.npy (480, 640, 3)
00727.npy (480, 640, 3)
00728.npy (480, 640, 3)
00729.npy (480, 640, 3)
00730.npy (480, 640, 3)
00731.npy (480, 640, 3)
00732.npy (480, 640, 3)
00733.npy (480, 640, 3)
00734.npy (480, 640, 3)
00735.npy (480, 640, 3)
00736.npy (480, 640, 3)
00737.npy (480, 640, 3)
00738.npy (480, 640, 3)
00739.npy (480, 640, 3)
00740.npy (480, 640, 3)
00741.npy (480, 640, 3)
00742.npy (480, 640, 3)
00743.npy (480, 640, 3)
00744.npy (480, 640, 3)
00745.npy (480, 640, 3)
00746.npy (480, 640, 3)
00747.npy (480, 640, 3)
00748.npy (480, 640, 3)
00749.npy (480, 640, 3)
00750.npy (480, 640, 3)
00751.npy (480, 640, 3)
00752.npy (480, 640, 3)
00753.npy (480, 640, 3)
00754.npy (480, 640, 3)
00755.npy (480, 640, 3)
00756.npy (480, 640, 3)
00757.npy (480, 640, 3)
00758.npy (480, 640, 3)
00759.npy (480, 640, 3)
00760.npy (480, 640, 3)
00761.npy (480, 640, 3)
00762.npy (480, 

# 저장한 pointcloud와 color array를 matching 합니다

In [42]:
def joint_pixelToPoint(path_PC, path_json):
    
    path_depth= path_PC
    path_json = path_json
    json_list = os.listdir(path_json) #path에 있는 json 리스트
    depth_list = os.listdir(path_depth)#path에 있는 depth array 리스트
    joint = {'r_shoulder' : ['1','2','3'], 'r_elbow' : ['2','3','4'], 'l_shoulder' : ['1','5','6'], 'l_elbow' : ['5','6','7']}
    matched = {}
    
    frame = 0
    matched = {}
    for j,k in zip(json_list, depth_list) :
        indicate=[]

        open_json = path_json +'//' + j
        open_depth = path_depth +'//' + k
        nf_depthData = np.load(open_depth)
        nf_depthData=nf_depthData.reshape(240,320,3)
        print('frame :',frame)
        print('json : ',j,'   /    Pointcloud : ',k)


        with open(open_json, encoding="utf-8") as data_file :
            data = json.load(data_file, object_pairs_hook=OrderedDict)

        for people, part_candidates in enumerate(data["part_candidates"]) :
            if people > 0 : print(", ", end="")
            for njoint in joint:

                try:
                    x = part_candidates[joint[njoint][1]][0]
                    pixel_x = int(x/2)
                except:
                    print("cannnot define",njoint, 'x')
                    continue

                try:
                    y = part_candidates[joint[njoint][1]][1]
                    pixel_y = int(y/2)
                except :
                    print("cannnot define",njoint, 'y')
                    continue
                print(njoint,' x: ',pixel_x,'y: ',pixel_y)
                    
                PC_point=nf_depthData[pixel_y][pixel_x]
                print(PC_point)
                
                indicate.append([njoint,PC_point])
                print()

            matched[frame]=[indicate]
        print('================frame end ==================')
        frame = frame + 1


    print("complete")
    return matched

In [44]:
# PCpath = 'C://Users//user//Desktop//openPose//outputs//pointcloud/'
# frame_list = sorted(os.listdir(PCpath)) 
# PCs=[]
# framecount=0
# for j in frame_list :#frame별 slice
#     open_frame = PCpath +'//' + j
#     save_load = np.load(open_frame)
#     npsavePath='C://Users//user//Desktop//openPose//outputs//pointcloud/'+str(framecount).zfill(5)
#     np.save(npsavePath,save_load)
#     framecount+=1

In [45]:
import os
import json
from collections import OrderedDict
import numpy as np
import math

path_PC = 'C://Users//user//Desktop//openPose//outputs//pointcloud/'
path_json = 'C://Users//user//Desktop//openPose//outputs//json/'

matched = joint_pixelToPoint(path_PC, path_json)

frame : 0
json :  raw_video_000000000000_keypoints.json    /    Pointcloud :  00000.npy
r_shoulder  x:  175 y:  137
[0. 0. 0.]

cannnot define r_elbow x
l_shoulder  x:  198 y:  145
[0. 0. 0.]

cannnot define l_elbow x
================frame end ==================
frame : 1
json :  raw_video_000000000001_keypoints.json    /    Pointcloud :  00001.npy
r_shoulder  x:  139 y:  127
[-0.0597642   0.02279878  0.565     ]

cannnot define r_elbow x
l_shoulder  x:  173 y:  133
[0. 0. 0.]

l_elbow  x:  179 y:  157
[0. 0. 0.]

================frame end ==================
frame : 2
json :  raw_video_000000000002_keypoints.json    /    Pointcloud :  00002.npy
r_shoulder  x:  133 y:  132
[-0.  0.  0.]

cannnot define r_elbow x
l_shoulder  x:  164 y:  132
[0. 0. 0.]

l_elbow  x:  169 y:  158
[0. 0. 0.]

================frame end ==================
frame : 3
json :  raw_video_000000000003_keypoints.json    /    Pointcloud :  00003.npy
r_shoulder  x:  133 y:  135
[-0.48387143  0.2885369   3.542     ]

ca

cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 35
json :  raw_video_000000000035_keypoints.json    /    Pointcloud :  00035.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 36
json :  raw_video_000000000036_keypoints.json    /    Pointcloud :  00036.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 37
json :  raw_video_000000000037_keypoints.json    /    Pointcloud :  00037.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 38
json :  raw_video_000000000038_keypoints.json    /    Pointcloud :  00038.npy
cannnot define r_shoulder x
cannnot defi

cannnot define l_elbow x
================frame end ==================
frame : 72
json :  raw_video_000000000072_keypoints.json    /    Pointcloud :  00072.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 73
json :  raw_video_000000000073_keypoints.json    /    Pointcloud :  00073.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 74
json :  raw_video_000000000074_keypoints.json    /    Pointcloud :  00074.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 75
json :  raw_video_000000000075_keypoints.json    /    Pointcloud :  00075.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
===============

cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 110
json :  raw_video_000000000110_keypoints.json    /    Pointcloud :  00110.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 111
json :  raw_video_000000000111_keypoints.json    /    Pointcloud :  00111.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 112
json :  raw_video_000000000112_keypoints.json    /    Pointcloud :  00112.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 113
json :  raw_video_000000000113_keypoints.json    /    Pointcloud :  00113.npy
cannnot define r_shoulder x
cannnot 

cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 146
json :  raw_video_000000000146_keypoints.json    /    Pointcloud :  00146.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
l_elbow  x:  174 y:  118
[ 0.0751149  -0.00597919  1.014     ]

================frame end ==================
frame : 147
json :  raw_video_000000000147_keypoints.json    /    Pointcloud :  00147.npy
cannnot define r_shoulder x
cannnot define r_elbow x
cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 148
json :  raw_video_000000000148_keypoints.json    /    Pointcloud :  00148.npy
r_shoulder  x:  258 y:  167
[0. 0. 0.]

cannnot define r_elbow x
l_shoulder  x:  177 y:  160
[0.09522158 0.22336504 1.064     ]

cannnot define l_elbow x
================frame end ==================
frame : 149
json :  raw_video_000000000

...

r_shoulder  x:  119 y:  58
[-0. -0.  0.]

r_elbow  x:  90 y:  90
[-0. -0.  0.]

l_shoulder  x:  183 y:  51
[ 0.36230242 -1.0544245   3.0110002 ]

l_elbow  x:  216 y:  74
[ 0.77549326 -0.6206007   2.6750002 ]

================frame end ==================
frame : 920
json :  raw_video_000000000920_keypoints.json    /    Pointcloud :  00920.npy
r_shoulder  x:  119 y:  58
[-0.75141186 -1.1321338   3.6030002 ]

r_elbow  x:  90 y:  90
[-0. -0.  0.]

l_shoulder  x:  183 y:  51
[ 0.3845628 -1.1192098  3.196    ]

l_elbow  x:  216 y:  75
[ 0.76244754 -0.5966459   2.63      ]

================frame end ==================
frame : 921
json :  raw_video_000000000921_keypoints.json    /    Pointcloud :  00921.npy
r_shoulder  x:  119 y:  58
[-0.7103272 -1.0702325  3.4060001]

r_elbow  x:  90 y:  90
[-0.5728345  -0.23994878  1.6020001 ]

l_shoulder  x:  182 y:  51
[ 0.33703882 -1.0246583   2.926     ]

l_elbow  x:  216 y:  75
[ 0.7891187 -0.6175171  2.7220001]

================frame end ==============

r_shoulder  x:  119 y:  58
[-0. -0.  0.]

r_elbow  x:  90 y:  90
[-1.2665291 -0.5305234  3.542    ]

l_shoulder  x:  183 y:  53
[ 0. -0.  0.]

l_elbow  x:  216 y:  74
[ 0. -0.  0.]

================frame end ==================
frame : 956
json :  raw_video_000000000956_keypoints.json    /    Pointcloud :  00956.npy
r_shoulder  x:  119 y:  58
[-0.7026108 -1.0586064  3.3690002]

r_elbow  x:  90 y:  90
[-1.428868  -0.5985239  3.9960003]

l_shoulder  x:  182 y:  53
[ 0.0593216  -0.17505538  0.51500005]

l_elbow  x:  216 y:  74
[ 0. -0.  0.]

================frame end ==================
frame : 957
json :  raw_video_000000000957_keypoints.json    /    Pointcloud :  00957.npy
r_shoulder  x:  118 y:  58
[-0.75240403 -1.1063677   3.5210001 ]

r_elbow  x:  90 y:  90
[-0. -0.  0.]

l_shoulder  x:  181 y:  53
[ 0.3298164 -1.0187204  2.9970002]

l_elbow  x:  216 y:  75
[ 0.77549326 -0.6068547   2.6750002 ]

================frame end ==================
frame : 958
json :  raw_video_000000000958_key

json :  raw_video_000000000987_keypoints.json    /    Pointcloud :  00987.npy
r_shoulder  x:  117 y:  58
[-0.7930369 -1.1387323  3.624    ]

r_elbow  x:  113 y:  109
[-0.38756275 -0.08442292  1.6190001 ]

l_shoulder  x:  177 y:  53
[ 0.2761784 -1.0489726  3.0860002]

l_elbow  x:  190 y:  101
[ 0.28399223 -0.16944407  1.817     ]

================frame end ==================
frame : 988
json :  raw_video_000000000988_keypoints.json    /    Pointcloud :  00988.npy
r_shoulder  x:  117 y:  58
[-0.7619631 -1.0941132  3.482    ]

r_elbow  x:  115 y:  109
[-0.38604462 -0.0878645   1.6850001 ]

l_shoulder  x:  175 y:  51
[ 0.24565046 -1.0859417   3.101     ]

l_elbow  x:  190 y:  99
[ 0.28821227 -0.19091356  1.8440001 ]

================frame end ==================
frame : 989
json :  raw_video_000000000989_keypoints.json    /    Pointcloud :  00989.npy
r_shoulder  x:  117 y:  58
[-0.7794695 -1.1192508  3.5620003]

r_elbow  x:  115 y:  109
[-0.8708342  -0.19820353  3.801     ]

l_shoulder  x: 

r_shoulder  x:  133 y:  39
[-0.2588753 -0.780466   1.8950001]

r_elbow  x:  128 y:  107
[-0.29149663 -0.11211089  1.7960001 ]

l_shoulder  x:  206 y:  38
[ 0.6637923 -1.1604948  2.7830002]

l_elbow  x:  224 y:  96
[ 0.842099   -0.30260485  2.5440001 ]

================frame end ==================
frame : 1021
json :  raw_video_000000001021_keypoints.json    /    Pointcloud :  01021.npy
r_shoulder  x:  134 y:  38
[-0.24466741 -0.77602607  1.8610001 ]

r_elbow  x:  128 y:  107
[-0.2940935  -0.11310965  1.812     ]

l_shoulder  x:  209 y:  38
[ 0.73285055 -1.2034451   2.8860002 ]

l_elbow  x:  228 y:  100
[ 0.8908749  -0.24932931  2.5340002 ]

================frame end ==================
frame : 1022
json :  raw_video_000000001022_keypoints.json    /    Pointcloud :  01022.npy
r_shoulder  x:  135 y:  38
[-0.44241545 -1.4603134   3.502     ]

r_elbow  x:  128 y:  107
[-0.5714697  -0.21978977  3.5210001 ]

l_shoulder  x:  211 y:  37
[ 0. -0.  0.]

l_elbow  x:  229 y:  95
[ 0. -0.  0.]

====

r_shoulder  x:  190 y:  68
[ 0.43497545 -0.7314631   2.7830002 ]

r_elbow  x:  164 y:  98
[ 0.03995819 -0.19136965  1.761     ]

l_shoulder  x:  234 y:  70
[ 0.9273223  -0.61244625  2.4250002 ]

cannnot define l_elbow x
================frame end ==================
frame : 1053
json :  raw_video_000000001053_keypoints.json    /    Pointcloud :  01053.npy
cannnot define r_shoulder x
r_elbow  x:  164 y:  108
[ 0.03875558 -0.09784077  1.7080001 ]

cannnot define l_shoulder x
cannnot define l_elbow x
================frame end ==================
frame : 1054
json :  raw_video_000000001054_keypoints.json    /    Pointcloud :  01054.npy
r_shoulder  x:  194 y:  68
[ 0.4898806  -0.72804624  2.7700002 ]

r_elbow  x:  167 y:  105
[ 0.06615337 -0.12620717  1.7360001 ]

l_shoulder  x:  265 y:  86
[ 1.283832   -0.40369546  2.3700001 ]

cannnot define l_elbow x
================frame end ==================
frame : 1055
json :  raw_video_000000001055_keypoints.json    /    Pointcloud :  01055.npy
r_shou

In [56]:
matched[1][0][0][0]

'r_shoulder'

# 각도 계산하기

In [ ]:
def get3Dangle(matched):
    
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    
    if (acute == True):
        return angle
    else:
        return 2 * np.pi - angle